<a href="https://colab.research.google.com/github/gyaneshhere/LLM-Security-and-Observability/blob/main/LangFuse/Develop/Prompt%20Management/prompt_management_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
description: Example of Open Source Prompt Management for Langchain applications using Langfuse.
category: Prompt Management
---

# Example: Langfuse Prompt Management with Langchain (Python)

[Langfuse Prompt Management](https://langfuse.com/docs/prompts) helps to version control and manage prompts collaboratively in one place. This example demostrates how to use prompts managed in Langchain applications.

_In addition, we use [Langfuse Tracing](https://langfuse.com/docs/tracing) via the native [Langchain integration](https://langfuse.com/docs/integrations/langchain) to inspect and debug the Langchain application._

## Setup

In [1]:
%pip install langfuse langchain langchain-openai --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.0/275.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52


In [2]:
from google.colab import userdata


In [3]:
import os

# get keys for your project from https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = userdata.get('LANGFUSE_PUBLIC_KEY')
os.environ["LANGFUSE_SECRET_KEY"] = userdata.get('LANGFUSE_SECRET_KEY')
os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com"

# your openai key
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [4]:
from langfuse import Langfuse
from langfuse.callback import CallbackHandler

# Initialize Langfuse client (prompt management)
langfuse = Langfuse()

# Initialize Langfuse CallbackHandler for Langchain (tracing)
langfuse_callback_handler = CallbackHandler()

# Optional, verify that Langfuse is configured correctly
assert langfuse.auth_check()
assert langfuse_callback_handler.auth_check()

## Add prompt to Langfuse Prompt Management

We add the prompt used in this example via the SDK. Alternatively, you can also edit and version the prompt in the Langfuse UI.

- `Name` that identifies the prompt in Langfuse Prompt Management
- Prompt with prompt template incl. `{{input variables}}`
- Config including `model_name` and `temperature`
- `labels` to include `production` to immediately use prompt as the default

In [5]:
langfuse.create_prompt(
    name="event-planner",
    prompt=
    "Plan an event titled {{Event Name}}. The event will be about: {{Event Description}}. "
    "The event will be held in {{Location}} on {{Date}}. "
    "Consider the following factors: audience, budget, venue, catering options, and entertainment. "
    "Provide a detailed plan including potential vendors and logistics.",
    config={
        "model":"gpt-4o",
        "temperature": 0,
    },
    labels=["production"]
);

Prompt in Langfuse UI

![Created prompt in Langfuse UI](https://langfuse.com/images/docs/prompt-management-langchain-prompt.png)

## Example application

### Get current prompt version from Langfuse

In [6]:
# Get current production version of prompt
langfuse_prompt = langfuse.get_prompt("event-planner")

```python
print(langfuse_prompt.prompt)
```

```
Plan an event titled {{Event Name}}. The event will be about: {{Event Description}}. The event will be held in {{Location}} on {{Date}}. Consider the following factors: audience, budget, venue, catering options, and entertainment. Provide a detailed plan including potential vendors and logistics.
 ```

In [7]:
print(langfuse_prompt.prompt)

Plan an event titled {{Event Name}}. The event will be about: {{Event Description}}. The event will be held in {{Location}} on {{Date}}. Consider the following factors: audience, budget, venue, catering options, and entertainment. Provide a detailed plan including potential vendors and logistics.


### Transform into Langchain PromptTemplate

Use the utility method `.get_langchain_prompt()` to transform the Langfuse prompt into a string that can be used in Langchain.

Context: Langfuse declares input variables in prompt templates using double brackets (`{{input variable}}`). Langchain uses single brackets for declaring input variables in PromptTemplates (`{input variable}`). The utility method `.get_langchain_prompt()` replaces the double brackets with single brackets.

Also, pass the Langfuse prompt as metadata to the PromptTemplate to automatically link generations that use the prompt.

In [8]:
from langchain_core.prompts import ChatPromptTemplate

langchain_prompt = ChatPromptTemplate.from_template(
        langfuse_prompt.get_langchain_prompt(),
        metadata={"langfuse_prompt": langfuse_prompt},
    )

Extract the configuration options from `prompt.config`

In [9]:
model = langfuse_prompt.config["model"]
temperature = str(langfuse_prompt.config["temperature"])
print(f"Prompt model configurations\nModel: {model}\nTemperature: {temperature}")

Prompt model configurations
Model: gpt-4o
Temperature: 0


### Create Langchain chain based on prompt

In [10]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model=model, temperature=temperature)

chain = langchain_prompt | model

## Invoke chain

In [11]:
example_input = {
    "Event Name": "Wedding",
    "Event Description": "The wedding of Julia and Alex, a charming couple who share a love for art and nature. This special day will celebrate their journey together with a blend of traditional and contemporary elements, reflecting their unique personalities.",
    "Location": "Central Park, New York City",
    "Date": "June 5, 2024"
}

In [12]:
# we pass the callback handler to the chain to trace the run in Langfuse
response = chain.invoke(input=example_input,config={"callbacks":[langfuse_callback_handler]})

print(response.content)

To plan an event titled "Green Future Festival," which focuses on sustainability, environmental awareness, and green technology, we will consider the following factors: audience, budget, venue, catering options, and entertainment. The event will be held in Central Park, New York City, on June 5, 2024.

### Event Overview
**Event Name:** Green Future Festival  
**Event Date:** June 5, 2024  
**Location:** Central Park, New York City  
**Event Description:** A day-long festival celebrating sustainability, featuring workshops, panel discussions, eco-friendly vendors, and live entertainment to promote environmental awareness and green technology.

### Audience
- **Target Audience:** Environmentally conscious individuals, families, students, professionals in green industries, and local community members.
- **Expected Attendance:** 5,000 - 7,000 people.

### Budget
- **Estimated Budget:** $150,000
  - Venue and Permits: $20,000
  - Catering: $30,000
  - Entertainment: $40,000
  - Marketing a

## View Trace in Langfuse

Now we can see that the trace incl. the prompt template have been logged to Langfuse

![Trace of prompt used in Langchain in Langfuse](https://langfuse.com/images/docs/prompt-management-langchain-trace.png)

## Iterate on prompt in Langfuse
We can now continue adapting our prompt template in the Langfuse UI and continuously update the prompt template in our Langchain application via the script above.